In [2]:
import pandas as pd
import sqlite3

## Подключение через `sqlite3`

In [3]:
data = sqlite3.connect("../data/checking-logs.sqlite")

## Схема таблицы test

In [4]:
pd.read_sql("PRAGMA table_info(checker)", data)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,status,TEXT,0,None,0
2,2,success,INTEGER,0,None,0
3,3,timestamp,TIMESTAMP,0,None,0
4,4,numTrials,INTEGER,0,None,0
5,5,labname,TEXT,0,None,0
6,6,uid,TEXT,0,None,0


## Первые 10 строк

In [5]:
pd.read_sql("SELECT * FROM test LIMIT 10;", data)

,index,uid,labname,first_commit_ts,first_view_ts
0,3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
1,4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
2,7,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
3,8,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
4,11,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447
5,18,user_19,laba04,2020-04-20 19:05:01.297780,2020-04-21 20:30:38.034966
6,19,user_25,laba04,2020-04-20 19:16:50.673054,2020-05-09 23:54:54.260791
7,20,user_21,laba04,2020-04-21 17:48:00.487806,2020-04-22 22:40:36.824081
8,21,user_30,project1,2020-04-22 12:36:24.053518,2020-04-17 22:46:26.785035
9,23,user_21,laba04s,2020-04-22 20:09:21.857747,2020-04-22 22:40:36.824081


## Найти минимум, максимум и среднее значение разницы в часах между первым коммитом и дедлайном лабораторной кроме `project1`, объединив с таблицей `deadlines`

- ### Минимум в `df_min` с `uid`

- ### максимум в `df_max` с `uid`

- ### среднее в `df_avg` без `uid`

---

- ### Найти среди всех `user` минимальное значение дельты между первым коммитом `user` и `deadline` лабораторной работы. Разрешен только один запрос

----

- `datetime(deadlines.deadlines, 'unixepoch')` -> Преобразует столбец `deadlines` из таблицы `deadlines` в формат даты и времени

- `(deadlines - unixepoch(first_commit_ts)) / 3600 AS delta` -> Вычисляет разницу во времени между `deadlines.deadlines` и временем первого коммита `first_commit_ts`. Разница делится на 3600, чтобы перевести секунды в часы, и результат именуется как `delta`

- `FROM test JOIN deadlines ON deadlines.labs = test.labname` -> Соединяет таблицу `test` с таблицей `deadlines` по условию, где столбец `labs` в таблице `deadlines` совпадает со столбцом `labname` в таблице `test`. Это связывает каждую *лабораторную* работу из таблицы `test` с соответствующим дедлайном из таблицы `deadlines`

- Функция `unixepoch` в *SQLite* преобразует дату и время в *Unix* временную метку

In [6]:
sql_request1 = """
SELECT uid, MIN(delta) AS min_deadline FROM (
                                             SELECT uid, first_commit_ts, DATETIME(deadlines.deadlines, 'unixepoch'),
                                                                                  (deadlines - unixepoch(first_commit_ts)) / 3600 AS delta
                                             FROM test JOIN deadlines ON deadlines.labs = test.labname
                                             WHERE labname != 'project1');
"""

df_min = pd.read_sql(sql_request1, data)

df_min

,uid,min_deadline
0,user_25,2


- ### Найти среди всех `user` максимальное значения дельты первым коммитом `user` и `deadline` лабораторной работы. Разрешен только один запрос

In [7]:
sql_request2 = """
SELECT uid, MAX(delta) as max_deadline FROM(
                                             SELECT uid, first_commit_ts, datetime(deadlines.deadlines, 'unixepoch'),
                                                                                  (deadlines - unixepoch(first_commit_ts)) / 3600 AS delta
                                             FROM test
                                             JOIN deadlines ON deadlines.labs = test.labname
                                             WHERE labname != 'project1') 

"""

df_max = pd.read_sql(sql_request2, data)

df_max

,uid,max_deadline
0,user_30,202


- ### Найти среди всех `user` среднее значения дельты первым коммитом `user` и `deadline` лабораторной работы. Разрешен только один запрос

In [8]:
sql_request3 = """
SELECT AVG(delta) AS avg_deadline FROM (
                                        SELECT uid, first_commit_ts, DATETIME(deadlines.deadlines, 'unixepoch'),
                                                                             (deadlines - unixepoch(first_commit_ts)) / 3600 AS delta
                                        FROM test
                                        JOIN deadlines ON deadlines.labs = test.labname
                                        WHERE labname != 'project1') 
"""

df_avg = pd.read_sql(sql_request3, data)

df_avg

,avg_deadline
0,89.125


## Проверить гипотезу

- ### Вычислить корреляцию между количеством просмотров новостной ленты `pageviews` и средней разницей `avg_diff`

---

- `(SELECT COUNT(*) FROM pageviews WHERE test.uid = pageviews.uid) AS page_views`

    - подсчитывает количество строк в таблице `pageviews`, где `uid` совпадает с `uid` из таблицы `test`. Это количество просмотров страниц для каждого пользователя

- `FROM test JOIN deadlines ON deadlines.labs = test.labname`

    - Соединяет таблицу `test` с таблицей `deadlines`, где столбец `labs` в `deadlines` совпадает со столбцом `labname` в `test`. Это связывает каждую лабораторную работу с ее дедлайном

In [9]:
sql_request = """
SELECT uid, round(avg(delta), 2) AS avg_diff, MAX(page_views) AS pageviews FROM(
                                                                                SELECT uid, first_commit_ts, DATETIME(deadlines.deadlines, 'unixepoch'),
                                                                                (deadlines - unixepoch(first_commit_ts)) / 3600 AS delta,
                                                                                (SELECT COUNT(*) FROM pageviews 
                                                                                WHERE test.uid = pageviews.uid) AS page_views
                                                                                
                                                                                FROM test
                                                                                JOIN deadlines ON deadlines.labs = test.labname
                                                                                WHERE labname != 'project1')

GROUP BY uid
"""

views_diff = pd.read_sql(sql_request, data)


views_diff

,uid,avg_diff,pageviews
0,user_1,64.40,28
1,user_10,74.80,89
2,user_14,159.00,143
3,user_17,61.60,47
4,user_18,5.67,3
5,user_19,98.75,16
6,user_21,95.50,10
7,user_25,92.60,179
8,user_28,86.40,149
9,user_3,105.40,317


In [10]:
views_diff.set_index('uid', inplace = True)

views_diff

,avg_diff,pageviews
uid,,
user_1,64.40,28
user_10,74.80,89
user_14,159.00,143
user_17,61.60,47
user_18,5.67,3
user_19,98.75,16
user_21,95.50,10
user_25,92.60,179
user_28,86.40,149


- вычисляет корреляцию между столбцами в *DataFrame*

- `corr()` в *pandas* вычисляет попарную корреляцию между числовыми столбцами в *DataFrame*

In [11]:
views_diff.corr()

,avg_diff,pageviews
avg_diff,1.000000,0.279734
pageviews,0.279734,1.000000


## Закрываем соединение с БД

In [12]:
data.close()